<a href="https://colab.research.google.com/github/itsmeeeeeee/BERT_Probing/blob/main/extract_embedding_2m.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 30.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 37.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 76.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 76.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 35.8 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np
from ast import literal_eval
from transformers import BertTokenizer, BertModel, AutoTokenizer
import torch
from pathlib import Path
from typing import Dict

In [3]:
model_ckpt = "bert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
model = BertModel.from_pretrained(model_ckpt)


In [4]:
# Returns a list of lists of indices for each subtoken from the tokenizer, if more than one subtoken appears
def get_indexof_subtokens(tokenizer_output):
    result = []
    temp_liste = [tokenizer_output[0]]  # Start with the first word in the temporary list

    for i in range(1, len(tokenizer_output)):
        # If the current word has the same property as the previous word
        if tokenizer_output[i][:2] == "##" and tokenizer_output[i - 1][:2] == "##":
            temp_liste.append(i)



        else:
            if len(temp_liste) > 1:  # If the temporary list contains more than one word, add it to the result.
                result.append(temp_liste)
            temp_liste = [i]  # Start a new temporary list with the current word.

    # Add the last temporary list to the result if it contains more than one word
    if len(temp_liste) > 1:
        result.append(temp_liste)

    return result

ll = ['[CLS]', 'Celebration', 'Shooting', 'Turn', '##s', 'Wedding', 'Into', 'a', 'Fun', '##eral', 'in', 'Southern',
      'Gaza', 'Strip', 'As', '##ad', '1', '/', '20', '/', '2007', 'Gaza', '-', 'UP', '##I', 'The', 'cheer', '##s',
      'and', 'hail', '##s', 'of', 'happiness', 'at', 'a', 'wedding', 'in', 'Khan', 'You', '##nes', 'in', 'the',
      'southern', 'Gaza', 'Strip', 'turned', 'into', 'screams', 'and', 'moan', '##s', 'of', 'pain', 'after', 'one',
      'of', 'the', 'c', '##ele', '##bra', '##tors', 'lost', 'control', 'of', 'his', 'weapon', ',', 'from', 'which', 'a',
      'number', 'of', 'bullets', 'were', 'released', 'that', 'killed', 'the', 'groom', "'", 's', 'brother', 'and',
      'hit', 'three', 'other', 'relatives', 'of', 'his', ',', 'turning', 'the', 'wedding', 'into', 'a', 'funeral', 'in',
      'moments', 'c', '##ele', '##bra', '##tors', '[SEP]']

get_indexof_subtokens(ll)

[[58, 59, 60], [99, 100, 101]]

In [5]:

"""Returns three lists: one list with main words (only those tokenized into a single subword),
   one with subword related to the main word, and one list with tokenizer indices of the main words.
    All three lists are of the same length"""

def get_main_token_subtoken_embed_index(tokens):
    main_token = []
    subtoken = []
    embed_index_main_token = []

    for i in range(len(tokens) - 1):
        s = ""
        if "##" in tokens[i] and "##" not in tokens[i - 1] and "##" not in tokens[i + 1]:
            s += tokens[i - 1]
            subtoken.append(tokens[i])
            s += tokens[i]
            embed_index_main_token.append(i - 1)


        main_token.append(s.replace("##", ""))
    main_token = [x for x in main_token if x != ""]
    return main_token, subtoken, embed_index_main_token


ll = ['[CLS]', 'Celebration', 'Shooting', 'Turn', '##s', 'Wedding', 'Into', 'a', 'Fun', '##eral', 'in', 'Southern',
      'Gaza', 'Strip', 'As', '##ad', '1', '/', '20', '/', '2007', 'Gaza', '-', 'UP', '##I', 'The', 'cheer', '##s',
      'and', 'hail', '##s', 'of', 'happiness', 'at', 'a', 'wedding', 'in', 'Khan', 'You', '##nes', 'in', 'the',
      'southern', 'Gaza', 'Strip', 'turned', 'into', 'screams', 'and', 'moan', '##s', 'of', 'pain', 'after', 'one',
      'of', 'the', 'c', '##ele', '##bra', '##tors', 'lost', 'control', 'of', 'his', 'weapon', ',', 'from', 'which', 'a',
      'number', 'of', 'bullets', 'were', 'released', 'that', 'killed', 'the', 'groom', "'", 's', 'brother', 'and',
      'hit', 'three', 'other', 'relatives', 'of', 'his', ',', 'turning', 'the', 'wedding', 'into', 'a', 'funeral', 'in',
      'moments', 'c', '##ele', '##bra', '##tors', '[SEP]']
main_token, subtoken, embed_index = get_main_token_subtoken_embed_index(ll)
print(main_token)
print(subtoken)
print(embed_index)

['Turns', 'Funeral', 'Asad', 'UPI', 'cheers', 'hails', 'Younes', 'moans']
['##s', '##eral', '##ad', '##I', '##s', '##s', '##nes', '##s']
[3, 8, 14, 23, 26, 29, 38, 49]


In [6]:

"""Returns three lists: one list with main words (only those tokenized into multi subwords),
   one with list of subwords related to the main words, and one list with tokenizer indices of the main words.
    All three lists are of the same length"""

def get_main_tokens_subtokens_embed_index(tokenizer_output):
    index_subtokens = get_indexof_subtokens(tokenizer_output)
    main_words = []
    subwords = []
    embed_index = []
    s = ""
    for inds in index_subtokens:
        #print(".......",res[0]-1)
        s += tokenizer_output[inds[0] - 1]
        l = []
        for ind in inds:
            l.append(tokenizer_output[ind])

            s += tokenizer_output[ind]
        subwords.append(l)
        main_words.append(s.replace("##", ""))
        s = ""
        embed_index.append(inds[0] - 1)

    return main_words, subwords, embed_index

ll = ['[CLS]', 'Celebration', 'Shooting', 'Turn', '##s', 'Wedding', 'Into', 'a', 'Fun', '##eral', 'in', 'Southern',
      'Gaza', 'Strip', 'As', '##ad', '1', '/', '20', '/', '2007', 'Gaza', '-', 'UP', '##I', 'The', 'cheer', '##s',
      'and', 'hail', '##s', 'of', 'happiness', 'at', 'a', 'wedding', 'in', 'Khan', 'You', '##nes', 'in', 'the',
      'southern', 'Gaza', 'Strip', 'turned', 'into', 'screams', 'and', 'moan', '##s', 'of', 'pain', 'after', 'one',
      'of', 'the', 'c', '##ele', '##bra', '##tors', 'lost', 'control', 'of', 'his', 'weapon', ',', 'from', 'which', 'a',
      'number', 'of', 'bullets', 'were', 'released', 'that', 'killed', 'the', 'groom', "'", 's', 'brother', 'and',
      'hit', 'three', 'other', 'relatives', 'of', 'his', ',', 'turning', 'the', 'wedding', 'into', 'a', 'funeral', 'in',
      'moments', 'c', '##ele', '##bra', '##tors', '[SEP]']
main_words, subwords, embed_index = get_main_tokens_subtokens_embed_index(ll)
print(main_words)
print(subwords)  #teilwort,embed_index)
print(embed_index)

['celebrators', 'celebrators']
[['##ele', '##bra', '##tors'], ['##ele', '##bra', '##tors']]
[57, 98]


In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
#The data is read from Google Drive
file_path = "/content/drive/My Drive/embedding_data/final_endlabels_with_sentences_word_count.csv"

final_df = pd.read_csv(file_path)

<ipython-input-8-44c19300809e>:4: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  final_df = pd.read_csv(file_path)


In [9]:
"""
This sentence appears twice consecutively in the dataset and has the same sentence number.
This causes issues with indexing the words within a sentence. Refer to the function "find_index_2".Therefore, it is deleted.

"""

final_df[final_df["Word"]=="worst"][100:150]
final_df[(final_df.index == 5222032)]["Sentence"].tolist()[0]
f_d=final_df[(final_df["Sentence_no"] != 264185)]

In [10]:
# Filters out the unnecessary columns for further processing
select_columns = f_d[["Sentence_row","Word_count","Sentence", "Sentence_no", "Word", "POS", "end_labels"]]
select_columns

,Sentence_row,Word_count,Sentence,Sentence_no,Word,POS,end_labels
0,Celebration Shooting Turns Wedding Into a Fune...,82.0,"('Celebration', 'Shooting', 'Turns', 'Wedding'...",0,Celebration,PROPN,NaN
1,Celebration Shooting Turns Wedding Into a Fune...,82.0,"('Celebration', 'Shooting', 'Turns', 'Wedding'...",0,Shooting,PROPN,NaN
2,Celebration Shooting Turns Wedding Into a Fune...,82.0,"('Celebration', 'Shooting', 'Turns', 'Wedding'...",0,Turns,VERB,NaN
3,Celebration Shooting Turns Wedding Into a Fune...,82.0,"('Celebration', 'Shooting', 'Turns', 'Wedding'...",0,Wedding,NOUN,NaN
4,Celebration Shooting Turns Wedding Into a Fune...,82.0,"('Celebration', 'Shooting', 'Turns', 'Wedding'...",0,Into,ADP,NaN
...,...,...,...,...,...,...,...
5910736,Barak is facing mounting pressure ICH from wit...,44.0,"('Barak', 'is', 'facing', 'mounting', 'pressur...",307833,drop,VERB,NaN
5910737,Barak is facing mounting pressure ICH from wit...,44.0,"('Barak', 'is', 'facing', 'mounting', 'pressur...",307833,out,ADP,NaN
5910738,Barak is facing mounting pressure ICH from wit...,44.0,"('Barak', 'is', 'facing', 'mounting', 'pressur...",307833,of,ADP,NaN
5910739,Barak is facing mounting pressure ICH from wit...,44.0,"('Barak', 'is', 'facing', 'mounting', 'pressur...",307833,the,DET,NaN


In [11]:
select_columns.shape

(5910725, 7)

In [12]:
filtered_df = select_columns[(select_columns['Word_count'] < 500) & (select_columns['Word_count'] > 5)]

In [13]:
filtered_df.shape

(5765986, 7)

In [14]:
# In this step, only the first two million data points are used.

final_data_1_2000000=filtered_df[:2000000]


In [15]:
# Raw sentences are saved without duplicates

texts_1_2000000 = final_data_1_2000000["Sentence_row"].tolist()
texts_ohne_dublikate_1_2000000 = pd.Series(texts_1_2000000).dropna().drop_duplicates().tolist()
print(len(texts_ohne_dublikate_1_2000000))


15308


In [16]:
import time
start_time = time.time()


"""
  Extract the main information for the correct assignment of the embedding vector
  to each word in the respective sentence. It returns a list of main tokens, a list of subtokens,
  a list of indices for the assignment of the embedding vector, a list for all tokens, and a list
  with all contextualized vectors for a sentence.

"""


all_main_tokens_1_2000000 = []
all_subtokens_1_2000000 = []
all_embeded_index_1_2000000 = []
all_tokens_1_2000000 = []
all_contextualized_vectors_1_2000000= []
for ind, el in enumerate(texts_ohne_dublikate_1_2000000):

    tokenized_input = tokenizer(el)
    tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
    #print(el)
    main_tokens, subtokens, embed_index_main_tokens= get_main_tokens_subtokens_embed_index(tokens)
    main_token, subtoken, embed_index_main_token = get_main_token_subtoken_embed_index(tokens)
    main_tokens += main_token
    subtokens += subtoken
    embed_index_main_tokens += embed_index_main_token
    all_tokens_1_2000000.append(tokens)
    all_main_tokens_1_2000000.append(main_tokens)
    all_subtokens_1_2000000.append(subtokens)
    all_embeded_index_1_2000000.append(embed_index_main_tokens)
    input_vectors = []
    with torch.no_grad():
        outputs = model(torch.tensor(tokenized_input["input_ids"]).unsqueeze(0))
        contextualized_vectors = outputs.last_hidden_state
        for el in contextualized_vectors:
          all_contextualized_vectors_1_2000000.append(el)



    print(f"""##########################################################################
############################# new element {ind} ##############################
########################################################################""")
elapsed_time_seconds = time.time() - start_time
elapsed_time_minutes = elapsed_time_seconds / 60

print(f"Der Code hat {elapsed_time_minutes:.2f} Minuten gedauert.")

Die letzten 5000 Zeilen der Streamingausgabe wurden abgeschnitten.
########################################################################
##########################################################################
############################# new element 13642 ##############################
########################################################################
##########################################################################
############################# new element 13643 ##############################
########################################################################
##########################################################################
############################# new element 13644 ##############################
########################################################################
##########################################################################
############################# new element 13645 ##############################
#########################

In [17]:
# Initializes the three new columns

final_data_1_2000000['subword'] = None
final_data_1_2000000['word_index'] = None
final_data_1_2000000['contextualized_vectors'] = None



<ipython-input-17-4e73ae7df8e6>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_data_1_2000000['subword'] = None
<ipython-input-17-4e73ae7df8e6>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_data_1_2000000['word_index'] = None
<ipython-input-17-4e73ae7df8e6>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/us

In [18]:
"""
    We iterate through the dataframe using a for loop. For all rows that have the same sentence,
    we define a dictionary called 'word_already_seen', so that words that appear more than once
    in a sentence don't receive the same index,
    but rather a unique index.

"""


def find_index_2(curr_word: str, sub_tokens: list, already_seen: Dict):
    last_index = already_seen.get(curr_word, 0)
    curr_index = sub_tokens.index(curr_word, last_index + 1)
    already_seen[curr_word] = curr_index
    return curr_index

In [19]:
import traceback

"""This function is the main function for assigning 'subwords', word_index,
and contextualized vectors to each word in the dataframe."""


old_sentence_list = ""
counter = -1
hauptworter_fuer_current_satz = []
teilwoerter_fuer_current_satz = []
embed_index_fuer_current_satz = []
tokens_fuer_current_satz = []
previous_tokens = []
previous_sentence_no = None
word_already_seen = Dict

tensor_output = []
for index, row in final_data_1_2000000.iterrows():
    sentence_list = row["Sentence_row"]


     #Search for the index in 'texts_without_duplicates'. This is important because it corresponds to the
     #index for contextualized vectors, subwords, main_words, and EmbeddingIndex


    index_of_Sentence = texts_ohne_dublikate_1_2000000.index(sentence_list)
    tokens_fuer_current_satz = all_tokens_1_2000000[index_of_Sentence]

    hauptworter_fuer_current_satz =all_main_tokens_1_2000000[index_of_Sentence]
    teilwoerter_fuer_current_satz = all_subtokens_1_2000000[index_of_Sentence]
    embed_index_fuer_current_satz = all_embeded_index_1_2000000[index_of_Sentence]
    sentence_no_fuer_current_satz = row['Sentence_no']
    contextualized_vectors_fuer_current_satz = all_contextualized_vectors_1_2000000[index_of_Sentence]

    #When we come across a new sentence or the same sentence with a different sentence number,
    #the 'word_already_seen' is emptied


    if previous_tokens != tokens_fuer_current_satz or previous_sentence_no != sentence_no_fuer_current_satz:
        #print(f"changing from S_no:{previous_sentence_no} to-> S_no:{sentence_no_fuer_current_satz}")
        word_already_seen = dict()
    current_word = row['Word']
    index_of_word = -1
    if current_word in hauptworter_fuer_current_satz:
        index_of_hauptwort = hauptworter_fuer_current_satz.index(current_word)
        final_data_1_2000000.at[index, 'subword'] = teilwoerter_fuer_current_satz[index_of_hauptwort]
        index_of_word = embed_index_fuer_current_satz[index_of_hauptwort]
        final_data_1_2000000.at[index, "word_index"] = index_of_word

    elif current_word in tokens_fuer_current_satz:
        try:
            index_of_word = find_index_2(current_word, tokens_fuer_current_satz, word_already_seen)
        except Exception as e:
            print(f"""*****************************************************************************
********************************** index {index} *******************************************
*************************************************************************************""")
            #print(index)
            #print("current_word", current_word)
            #print("tokens_fuer_current_satz:")
            #print(tokens_fuer_current_satz)
            #print("word already seen")
            #print(word_already_seen)
            traceback.print_exc()
            break
        final_data_1_2000000.at[index, "word_index"] = index_of_word

    if index_of_word != -1:

        # If 'current_word' appears in main_word or in 'tokens', then the index must be a value that is >= 0

        e = contextualized_vectors_fuer_current_satz[index_of_word].tolist()
        tensor_output.append(e)

    else:
      # For the word (which was previously tokenized in our dataframe using spaCy), the tokenizer couldn't generate a value
        tensor_output.append(None)

    previous_tokens = tokens_fuer_current_satz
    previous_sentence_no = sentence_no_fuer_current_satz

    print("hallo there")

final_data_1_2000000['contextualized_vectors'] = tensor_output

# Filtered by end labels (only 0 and 1 labels are taken)
filtered_endlabels=final_data_1_2000000[(final_data_1_2000000["end_labels"] >= 0.0) & (final_data_1_2000000["end_labels"] <= 1.0)]

#  The NaN values from the "contextualized_vectors" are removed
filtered_endlabels.dropna(subset=["contextualized_vectors"],inplace=True)

# Due to the sentences repeating themselves in the dataset, similar embedding vectors are remove
end_embedding = filtered_endlabels.drop_duplicates(subset="contextualized_vectors")

#Save data
end_embedding.to_csv("/content/drive/My Drive/embedding_data/embedding_1_2000000.csv")
print("Fertig")

Die letzten 5000 Zeilen der Streamingausgabe wurden abgeschnitten.
hallo there
hallo there
hallo there
hallo there
hallo there
hallo there
hallo there
hallo there
hallo there
hallo there
hallo there
hallo there
hallo there
hallo there
hallo there
hallo there
hallo there
hallo there
hallo there
hallo there
hallo there
hallo there
hallo there
hallo there
hallo there
hallo there
hallo there
hallo there
hallo there
hallo there
hallo there
hallo there
hallo there
hallo there
hallo there
hallo there
hallo there
hallo there
hallo there
hallo there
hallo there
hallo there
hallo there
hallo there
hallo there
hallo there
hallo there
hallo there
hallo there
hallo there
hallo there
hallo there
hallo there
hallo there
hallo there
hallo there
hallo there
hallo there
hallo there
hallo there
hallo there
hallo there
hallo there
hallo there
hallo there
hallo there
hallo there
hallo there
hallo there
hallo there
hallo there
hallo there
hallo there
hallo there
hallo there
hallo there
hallo there
hallo the

<ipython-input-19-ff5732f28e31>:83: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_data_1_2000000['contextualized_vectors'] = tensor_output
<ipython-input-19-ff5732f28e31>:89: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_endlabels.dropna(subset=["contextualized_vectors"],inplace=True)


Fertig
